In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head(5)

In [ ]:
credits.head(1)['cast'].values

In [ ]:
credits.head(1)['crew'].values
# show 1 crew values

Marge 2 dataset

In [9]:
#dataset marge code

#movies.merge(credits,on='title')
#check size
# movies.merge(credits,on='title').shape
#assign new df in movies
movies = movies.merge(credits,on='title')

In [10]:
#check size
movies.shape
#credits.shape

(4809, 23)

DATA TRIMMING DROP UNWANTED COLUMNS OF MOVIES DATAFRAME


In [ ]:
#df info
movies.info()

In [12]:
#movies_id
#title
#genres
#keywords
#overview
#cast
#crew

In [13]:
#storing only prefered columns back into movies df
movies = movies[['movie_id','title','genres','overview','keywords','cast','crew']]

In [ ]:
#df info
movies.info()

In [ ]:
movies.head(1)

Data Preprocessing


In [ ]:
#check for missing data
movies.isnull().sum()

In [17]:
#Drop missing datas
movies.dropna(inplace=True)

In [19]:
#Check for duplicate data
movies.duplicated().sum()

0

In [ ]:
#Check data
movies.iloc[0].genres

Preprocessing



In [17]:
#GOAL
#'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
#['Action','Adventure','Fantacy']

In [ ]:
#Convert string of list to list 
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [26]:
#helper function
def converttoL(obj):
    L= []
    for i in ast.literal_eval(obj): #using ast literal_eval function from string = list
        L.append(i['name'])
    return L    

In [29]:
#converting genres to a list from string
movies['genres'] = movies['genres'].apply(converttoL) 

In [ ]:
movies.head(1)

In [31]:
#converting keywords to a list from string
movies['keywords'] = movies['keywords'].apply(converttoL) 

In [ ]:
movies.head(1)

In [34]:
#helper function
def convertcast(obj):
    L= []
    counter = 0
    for i in ast.literal_eval(obj): #using ast literal_eval function from string = list
        if counter !=4: #looping for 4 cast only
            L.append(i['name'])
            counter +=1
        else:
             break    
    return L    

In [35]:
#converting casts to a list from string
movies['cast'] = movies['cast'].apply(convertcast) 

In [ ]:
movies['cast'][0]

In [ ]:
movies.head()

In [38]:
#Helper function
def fDirector(obj):
    L= []
    for i in ast.literal_eval(obj): #using ast literal_eval function from string = list
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L    

In [39]:
#converting casts (Director) to a list from string
movies['crew'] = movies['crew'].apply(fDirector)

In [ ]:
movies.head(1)

overview

In [ ]:
movies['overview'][0]
#its a string

In [42]:
# spliting the overview data into words with lambda split function
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head(1)

Removing Spaces from data

In [44]:
#Science Fiction to ScienceFiction
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head(5)

Adding all data to tags

In [47]:
movies['tags']= movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
movies['tags'][0]

NEW DATAFRAME

In [50]:
nFrame = movies[['movie_id','title','tags']]

In [ ]:
nFrame

Converting Tags to string

In [ ]:
#Converting Tags to string
nFrame['tags'] = nFrame['tags'].apply(lambda x:" ".join(x))

In [ ]:
nFrame.head()

In [ ]:
nFrame['tags'][0] #overview of 0th data

In [ ]:
#convert to lowercase
nFrame['tags'] = nFrame['tags'].apply(lambda x:x.lower())

In [ ]:
nFrame.head()

Preprocessing Complete

Converting data to vector

In [48]:
#remove stopwords like a an the and or 
#making vector for 5000 words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [50]:
#converting nFrame ['tags'] to vector
#cv.fit_transform(nFrame['tags']).toarray().shape
vectors = cv.fit_transform(nFrame['tags']).toarray()


In [ ]:
cv.get_feature_names_out()
#len(cv.get_feature_names_out())

In [52]:
import nltk
#import nltk NLP library to stem words

In [53]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [55]:
#sample
ps.stem('believed')

'believ'

In [56]:
#helper function
def wordRoot(text):
    w=[]
    for i in text.split():
        w.append(ps.stem(i))
    return " ".join(w)


In [ ]:
#test
wordRoot('hello im loving this system')

In [ ]:
nFrame['tags'] = nFrame['tags'].apply(wordRoot)

Calculating Movie Vector cosine distaces

In [59]:
#Calculating Movie Vector cosine distaces
from sklearn.metrics.pairwise import cosine_similarity

In [60]:
#cosine_similarity(vectors).shape
match = cosine_similarity(vectors)

In [61]:
#Similarity Score
match[0]

array([1.        , 0.08964215, 0.05884899, ..., 0.02548236, 0.02817181,
       0.        ])

In [ ]:
#Recomendation for movie index 0
sorted(list(enumerate(match[0])), reverse=True, key=lambda x:x[1])[1:6]

In [ ]:
#Fetch Index of movie
nFrame[nFrame['title'] == 'Titan A.E.'].index[0]

In [65]:
#fetch title from index
nFrame.iloc[539].title


'Titan A.E.'

In [60]:
def recommend(movie):
    movie_index = nFrame[nFrame['title'] == movie].index[0]
    distances = match[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(nFrame.iloc[i[0]].title)
        #print(i[0])
    
 

In [ ]:
recommend('Batman Begins')